In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [25]:
customers = pd.read_csv("Ecommerce/olist_customers_dataset.csv")
order_items = pd.read_csv("Ecommerce/olist_order_items_dataset.csv")
orders = pd.read_csv("Ecommerce/olist_orders_dataset.csv")
products = pd.read_csv("Ecommerce/olist_products_dataset.csv")

orders.head(50)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01 00:00:00
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07 00:00:00
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06 00:00:00
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23 00:00:00


In [ ]:
# 데이터 타입 변환
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp']) # 주문 시간
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at']) # 주문 승인 시간
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date']) # 배송사 출발 시간
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date']) # 배송 완료 시간
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date']) # 예상시간

# 배송 소요일
orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']

# 배송 지연일
orders['order_estimated_delivery_date'] - orders['order_delivered_customer_date']
# 여기서... 음수면... 오래 걸린거......

0        7 days 02:34:47
1        5 days 08:32:15
2       17 days 05:53:31
3       12 days 23:31:18
4        9 days 05:42:58
              ...       
99436   10 days 08:51:59
99437    1 days 06:22:04
99438    5 days 12:35:43
99439   20 days 00:27:06
99440   17 days 10:51:30
Length: 99441, dtype: timedelta64[ns]